In [2]:
import numpy as np
import pandas as pd
from typing import Dict, List
import logging
import ipywidgets as widgets
from IPython.display import display, clear_output  # Ensure clear_output is imported
import plotly.express as px

# Setup audit trail logger
logging.basicConfig(filename='frtb_audit.log', level=logging.INFO, format='%(asctime)s %(message)s')

# -----------------------------
# Regulatory Parameters Loader
# -----------------------------
class RegulatoryParameters:
    def __init__(self):
        self.risk_weights = self.load_risk_weights()
        self.correlations = self.load_correlations()

    def load_risk_weights(self) -> Dict[str, Dict[str, float]]:
        return {
            'IR': { '1Y': 0.005, '2Y': 0.0055, '3Y': 0.0056, '5Y': 0.006, '7Y': 0.0062, '10Y': 0.0065 },
            'EQ': { 'Tech': 0.08, 'Energy': 0.07, 'Finance': 0.07, 'Healthcare': 0.075 },
            'FX': { 'USD': 0.06, 'EUR': 0.05, 'GBP': 0.055, 'JPY': 0.05 },
            'VEGA': { 'IR': 0.005, 'EQ': 0.07, 'FX': 0.06 },
            'CURVATURE': { 'IR': 0.003, 'EQ': 0.06, 'FX': 0.05 },
        }

    def load_correlations(self) -> Dict[str, pd.DataFrame]:
        return {
            'IR': pd.DataFrame(
                [[1.0, 0.98, 0.95, 0.92, 0.88, 0.85], [0.98, 1.0, 0.96, 0.94, 0.91, 0.89],
                 [0.95, 0.96, 1.0, 0.96, 0.92, 0.90], [0.92, 0.94, 0.96, 1.0, 0.94, 0.92],
                 [0.88, 0.91, 0.92, 0.94, 1.0, 0.95], [0.85, 0.89, 0.90, 0.92, 0.95, 1.0]],
                index=['1Y', '2Y', '3Y', '5Y', '7Y', '10Y'], columns=['1Y', '2Y', '3Y', '5Y', '7Y', '10Y']
            ),
            'EQ': pd.DataFrame(
                [[1.0, 0.75, 0.72, 0.70], [0.75, 1.0, 0.78, 0.76], [0.72, 0.78, 1.0, 0.74], [0.70, 0.76, 0.74, 1.0]],
                index=['Tech', 'Energy', 'Finance', 'Healthcare'], columns=['Tech', 'Energy', 'Finance', 'Healthcare']
            ),
            'FX': pd.DataFrame(
                [[1.0, 0.6, 0.65, 0.58], [0.6, 1.0, 0.55, 0.52], [0.65, 0.55, 1.0, 0.61], [0.58, 0.52, 0.61, 1.0]],
                index=['USD', 'EUR', 'GBP', 'JPY'], columns=['USD', 'EUR', 'GBP', 'JPY']
            ),
            'VEGA': pd.DataFrame(
                [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
                index=['IR', 'EQ', 'FX'], columns=['IR', 'EQ', 'FX']
            ),
            'CURVATURE': pd.DataFrame(
                [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
                index=['IR', 'EQ', 'FX'], columns=['IR', 'EQ', 'FX']
            )
        }

# --------------------------
# Sensitivity-based Method
# --------------------------
class SensitivityBasedMethod:
    def __init__(self, sensitivities: pd.DataFrame, params: RegulatoryParameters):
        self.sensitivities = sensitivities
        self.params = params

    def compute_risk_class_charge(self, risk_class: str) -> float:
        df = self.sensitivities[self.sensitivities['risk_class'] == risk_class]
        risk_weights = self.params.risk_weights[risk_class]
        correlations = self.params.correlations[risk_class]

        buckets = df['bucket'].unique()
        bucket_charges = []

        for b in buckets:
            d = df[df['bucket'] == b]
            s = np.array([d[d['bucket_label'] == label]['sensitivity'].sum() for label in correlations.columns])
            rw = np.array([risk_weights[label] for label in correlations.columns])
            weighted = rw * s
            corr_matrix = correlations.to_numpy()
            Kb = np.sqrt(weighted @ corr_matrix @ weighted.T)
            logging.info(f"{risk_class} bucket {b}: weighted={weighted}, Kb={Kb}")
            bucket_charges.append(Kb)

        total_charge = np.sqrt(np.sum([x ** 2 for x in bucket_charges]))
        logging.info(f"{risk_class} total capital charge: {total_charge}")
        return total_charge

# -------------------
# Default Risk Charge
# -------------------
class DefaultRiskCharge:
    def __init__(self, jtd_data: pd.DataFrame):
        self.jtd_data = jtd_data

    def compute(self) -> float:
        self.jtd_data['DRC'] = self.jtd_data['exposure'] * self.jtd_data['PD'] * self.jtd_data['LGD']
        total_drc = self.jtd_data['DRC'].sum()
        logging.info(f"Default Risk Charge: {total_drc}")
        return total_drc

# --------------------------
# Residual Risk Add-On
# --------------------------
class ResidualRiskAddOn:
    def __init__(self, residual_data: pd.DataFrame):
        self.residual_data = residual_data

    def compute(self) -> float:
        self.residual_data['RRAO'] = self.residual_data['notional'] * self.residual_data['add_on_factor']
        total_rrao = self.residual_data['RRAO'].sum()
        logging.info(f"Residual Risk Add-On: {total_rrao}")
        return total_rrao

# --------------------------
# FRTB SA Master Calculator
# --------------------------
class FRTBStandardizedApproach:
    def __init__(self, sensitivities: pd.DataFrame, jtd_data: pd.DataFrame, residual_data: pd.DataFrame):
        self.params = RegulatoryParameters()
        self.sbm = SensitivityBasedMethod(sensitivities, self.params)
        self.drc = DefaultRiskCharge(jtd_data)
        self.rrao = ResidualRiskAddOn(residual_data)

    def compute_total_capital(self) -> Dict[str, float]:
        capital = {}
        for risk_class in ['IR', 'EQ', 'FX']:
            capital[f"SBM_{risk_class}"] = self.sbm.compute_risk_class_charge(risk_class)
        capital['VEGA'] = self.sbm.compute_risk_class_charge('VEGA') if 'VEGA' in self.params.risk_weights else 0
        capital['CURVATURE'] = self.sbm.compute_risk_class_charge('CURVATURE') if 'CURVATURE' in self.params.risk_weights else 0
        capital['DRC'] = self.drc.compute()
        capital['RRAO'] = self.rrao.compute()
        capital['Total_Capital'] = sum(capital.values())
        return capital

    def report(self) -> pd.DataFrame:
        capital_dict = self.compute_total_capital()
        df_report = pd.DataFrame.from_dict(capital_dict, orient='index', columns=['Capital'])
        df_report.index.name = 'Component'
        return df_report
    


    # -------------------------
    # Value at Risk (VaR)
    # -------------------------
    def compute_var(self, portfolio_returns: pd.Series, alpha: float = 0.01) -> float:
        # VaR calculation using historical simulation
        var = portfolio_returns.quantile(alpha)
        logging.info(f"VaR (at {alpha*100}% confidence level): {var}")
        return var

    # ---------------------------
    # Expected Shortfall (ES)
    # ---------------------------
    def compute_es(self, portfolio_returns: pd.Series, alpha: float = 0.01) -> float:
        # ES is the average of the worst losses beyond the VaR threshold
        var = self.compute_var(portfolio_returns, alpha)
        es = portfolio_returns[portfolio_returns <= var].mean()
        logging.info(f"Expected Shortfall (at {alpha*100}% confidence level): {es}")
        return es

    # -------------------------
    # Stress Testing
    # -------------------------
    def stress_test(self, portfolio_returns: pd.Series, shock: float = 0.05) -> float:
        # Applying stress scenario: a 5% drop in returns
        stressed_returns = portfolio_returns * (1 - shock)
        stressed_value = stressed_returns.sum()
        logging.info(f"Stress test (5% shock): {stressed_value}")
        return stressed_value    

# -------------------
# Example Data Inputs
# -------------------
sensitivities = pd.DataFrame([
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '1Y', 'sensitivity': 100000},
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '2Y', 'sensitivity': 200000},
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '3Y', 'sensitivity': 150000},
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '5Y', 'sensitivity': 120000},
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '7Y', 'sensitivity': 110000},
    {'risk_class': 'IR', 'bucket': 'IR', 'bucket_label': '10Y', 'sensitivity': 100000},
    {'risk_class': 'EQ', 'bucket': 'EQ', 'bucket_label': 'Tech', 'sensitivity': 120000},
    {'risk_class': 'EQ', 'bucket': 'EQ', 'bucket_label': 'Energy', 'sensitivity': 90000},
    {'risk_class': 'EQ', 'bucket': 'EQ', 'bucket_label': 'Finance', 'sensitivity': 95000},
    {'risk_class': 'EQ', 'bucket': 'EQ', 'bucket_label': 'Healthcare', 'sensitivity': 115000},
    {'risk_class': 'FX', 'bucket': 'FX', 'bucket_label': 'USD', 'sensitivity': 50000},
    {'risk_class': 'FX', 'bucket': 'FX', 'bucket_label': 'EUR', 'sensitivity': 40000},
    {'risk_class': 'FX', 'bucket': 'FX', 'bucket_label': 'GBP', 'sensitivity': 45000},
    {'risk_class': 'FX', 'bucket': 'FX', 'bucket_label': 'JPY', 'sensitivity': 38000},
    {'risk_class': 'VEGA', 'bucket': 'IR', 'bucket_label': 'IR', 'sensitivity': 50000},
    {'risk_class': 'VEGA', 'bucket': 'EQ', 'bucket_label': 'EQ', 'sensitivity': 30000},
    {'risk_class': 'VEGA', 'bucket': 'FX', 'bucket_label': 'FX', 'sensitivity': 40000},
    {'risk_class': 'CURVATURE', 'bucket': 'IR', 'bucket_label': 'IR', 'sensitivity': 25000},
    {'risk_class': 'CURVATURE', 'bucket': 'EQ', 'bucket_label': 'EQ', 'sensitivity': 20000},
    {'risk_class': 'CURVATURE', 'bucket': 'FX', 'bucket_label': 'FX', 'sensitivity': 18000},

])

jtd_data = pd.DataFrame([
    {'exposure': 1000000, 'PD': 0.01, 'LGD': 0.4},
    {'exposure': 2000000, 'PD': 0.02, 'LGD': 0.3},
    {'exposure': 1500000, 'PD': 0.015, 'LGD': 0.35}
])

residual_data = pd.DataFrame([
    {'notional': 500000, 'add_on_factor': 0.25},
    {'notional': 700000, 'add_on_factor': 0.3},
    {'notional': 600000, 'add_on_factor': 0.2}
])

# -------------------
# Final Run
# -------------------

# Portfolio returns data (for VaR and ES calculations)
portfolio_returns = pd.Series(np.random.normal(0.01, 0.02, 10000))  # Simulated returns

model = FRTBStandardizedApproach(sensitivities, jtd_data, residual_data)
capital_results = model.report()
print(capital_results.to_string(formatters={'Capital': '{:,.2f}'.format}))

# Compute VaR and ES
var = model.compute_var(portfolio_returns)
es = model.compute_es(portfolio_returns)

# Stress Testing
stressed_value = model.stress_test(portfolio_returns)

# Display Results
print(f"Value at Risk (VaR): {var}")
print(f"Expected Shortfall (ES): {es}")
print(f"Stress Test Value: {stressed_value}")

# -------------------
# Interactive Widgets and Charts
# -------------------
# Dropdown for component selection
component_dropdown = widgets.Dropdown(
    options=capital_results.index.tolist(),
    value='Total_Capital',
    description='Component:',
    style={'description_width': 'initial'}
)

# Dropdown for risk class and bucket breakdown
risk_class_dropdown = widgets.Dropdown(
    options=['IR', 'EQ', 'FX', 'VEGA', 'CURVATURE'],
    value='IR',
    description='Risk Class:',
    style={'description_width': 'initial'}
)

bucket_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Bucket:',
    style={'description_width': 'initial'}
)

# Output area
output = widgets.Output()

# Update bucket options based on risk class
def update_buckets(change=None):
    risk_class = risk_class_dropdown.value
    unique_buckets = sensitivities[sensitivities['risk_class'] == risk_class]['bucket'].unique()
    bucket_dropdown.options = unique_buckets
    bucket_dropdown.value = unique_buckets[0]  # Set the first bucket as the default

# Plot function for breakdown
def update_chart(change=None):
    with output:
        clear_output(wait=True)  # Clears previous outputs before displaying new ones
        selected_component = component_dropdown.value
        selected_risk_class = risk_class_dropdown.value
        selected_bucket = bucket_dropdown.value
        
        # If "Total", show pie of all components
        if selected_component == 'Total_Capital':
            fig = px.pie(
                capital_results.reset_index(), 
                names='Component', 
                values='Capital', 
                title="Total Capital Breakdown"
            )
        else:
            fig = px.bar(
                sensitivities[sensitivities['risk_class'] == selected_risk_class],
                x='bucket_label', y='sensitivity',
                title=f"{selected_risk_class} Bucket Sensitivity Breakdown"
            )
        fig.show()

# Connect the widgets with their update functions
component_dropdown.observe(update_chart, names='value')
risk_class_dropdown.observe(update_buckets, names='value')
bucket_dropdown.observe(update_chart, names='value')

# Initializing the chart
update_buckets()
update_chart()

# Display the interactive components
display(component_dropdown, risk_class_dropdown, bucket_dropdown, output)


                 Capital
Component               
SBM_IR          4,357.57
SBM_EQ         27,961.92
SBM_FX          7,831.61
VEGA            3,198.83
CURVATURE       1,501.87
DRC            23,875.00
RRAO          455,000.00
Total_Capital 523,726.81
Value at Risk (VaR): -0.0380483999751197
Expected Shortfall (ES): -0.04584818183448123
Stress Test Value: 92.76508262549822


Dropdown(description='Component:', index=7, options=('SBM_IR', 'SBM_EQ', 'SBM_FX', 'VEGA', 'CURVATURE', 'DRC',…

Dropdown(description='Risk Class:', options=('IR', 'EQ', 'FX', 'VEGA', 'CURVATURE'), style=DescriptionStyle(de…

Dropdown(description='Bucket:', options=('IR',), style=DescriptionStyle(description_width='initial'), value='I…

Output()